In [50]:
import numpy as np
import pandas as pd
import duckdb

In [51]:
from datasets.diabetes import Diabetes

ds = Diabetes("../data/diabetes")

In [52]:
train_ds = ds.train_data()

In [53]:
train_ds[0][0].shape

torch.Size([21])

In [54]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [55]:
model = nn.Sequential(
    nn.Linear(21, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

In [56]:
loss_fn = nn.BCELoss()
optimizer = AdamW(model.parameters())

In [57]:
train_dl = DataLoader(train_ds, batch_size=64)

In [59]:
model.train()
for epoch in range(100):
    epoch_loss = 0
    for X, y in train_dl:
        model.zero_grad()
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()
        
    epoch_loss /= len(train_dl)
    print(f"Epoch {epoch}: loss={epoch_loss:.4f}")

tensor([[0.6330],
        [0.0374],
        [0.3296],
        [0.1519],
        [0.1432],
        [0.1349],
        [0.2761],
        [0.2832],
        [0.3996],
        [0.1128],
        [0.0998],
        [0.4870],
        [0.1006],
        [0.4091],
        [0.1928],
        [0.0718],
        [0.2308],
        [0.0417],
        [0.0033],
        [0.0094],
        [0.2455],
        [0.5734],
        [0.0751],
        [0.0886],
        [0.3650],
        [0.0150],
        [0.6076],
        [0.4494],
        [0.4167],
        [0.0795],
        [0.5759],
        [0.3195],
        [0.0041],
        [0.3344],
        [0.2147],
        [0.1883],
        [0.3524],
        [0.0308],
        [0.0149],
        [0.4147],
        [0.0114],
        [0.3199],
        [0.1931],
        [0.0476],
        [0.1144],
        [0.0930],
        [0.0941],
        [0.2108],
        [0.5045],
        [0.4718],
        [0.5424],
        [0.1752],
        [0.4530],
        [0.0957],
        [0.0386],
        [0

KeyboardInterrupt: 

In [ ]:
model.eval()
test_loss = 0
test_corr = 0

for X, y in train_dl:
    with torch.no_grad():        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        
        test_loss += loss.cpu().item()
        test_corr += ((y_pred > 0.5) == y).sum()
            

test_loss /= len(train_dl)
test_acc = test_corr / len(train_dl.dataset)

In [ ]:
test_loss

In [ ]:
test_acc